In [1]:
import argparse
import copy
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
import torch.distributed as dist
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import get_git_hash

from mmdet import __version__
from mmdet.apis import init_random_seed, set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import (collect_env, get_device, get_root_logger,
                         setup_multi_processes, update_data_root)

import wandb

In [2]:
cfg = Config.fromfile('./swin_faster_cfg.py')

cfg.device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
dataset = build_dataset(cfg.data.train)
dataset

loading annotations into memory...
Done (t=3.31s)
creating index...
index created!



CocoDataset Train dataset with number of images 41820, and instance counts: 
+----------+--------+----------------+-------+--------------+-------+----------+-------+----------+-------+
| category | count  | category       | count | category     | count | category | count | category | count |
+----------+--------+----------------+-------+--------------+-------+----------+-------+----------+-------+
|          |        |                |       |              |       |          |       |          |       |
| 0 [선박] | 431471 | 1 [기타부유물] | 1773  | 2 [어망부표] | 20695 |          |       |          |       |
+----------+--------+----------------+-------+--------------+-------+----------+-------+----------+-------+

In [4]:
dataset.pipeline

Compose(
    LoadImageFromFile(to_float32=False, color_type='color', channel_order='bgr', file_client_args={'backend': 'disk'})
    LoadAnnotations(with_bbox=True, with_label=True, with_mask=False, with_seg=False, poly2mask=True, poly2mask={'backend': 'disk'})
    Resize(img_scale=[(768, 432)], multiscale_mode=range, ratio_range=None, keep_ratio=True, bbox_clip_border=True)
    RandomFlip(flip_ratio=0.0)
    Normalize(mean=[123.675 116.28  103.53 ], std=[58.395 57.12  57.375], to_rgb=True)
    Pad(size=None, size_divisor=32, pad_to_square=False, pad_val={'img': 0, 'masks': 0, 'seg': 255})
    DefaultFormatBundle(img_to_float=True)
    Collect(keys=['img', 'gt_bboxes', 'gt_labels'], meta_keys=('filename', 'ori_filename', 'ori_shape', 'img_shape', 'pad_shape', 'scale_factor', 'flip', 'flip_direction', 'img_norm_cfg'))
)

In [5]:
import numpy as np

i = np.random.randint(0, len(dataset))
print(i)

dataset[i]

28920


{'img_metas': DataContainer({'filename': '../data/aihub/train/images/20201224/0615/PYS_SN_20201224_0615_0045.jpg', 'ori_filename': 'train/images/20201224/0615/PYS_SN_20201224_0615_0045.jpg', 'ori_shape': (2160, 3840, 3), 'img_shape': (432, 768, 3), 'pad_shape': (448, 768, 3), 'scale_factor': array([0.2, 0.2, 0.2, 0.2], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}),
 'img': DataContainer(tensor([[[0.4851, 0.4851, 0.4851,  ..., 0.4166, 0.4166, 0.4166],
          [0.4851, 0.4851, 0.4851,  ..., 0.4166, 0.4166, 0.4166],
          [0.4851, 0.4851, 0.5022,  ..., 0.4508, 0.4508, 0.4508],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.6954, 0.6954, 0.6954,  ...

In [6]:
import matplotlib.pyplot as plt
import matplotlib as mpl

img = dataset[i]['img'].data.numpy()

min_ = np.min(img)
max_ = np.max(img)

img = img - (max_+min_)/2
img = img / (max_-min_)
img = img + 0.5

img = np.transpose(img, (1, 2, 0))
img = img[0:cfg.img_scale[1], 0:cfg.img_scale[0]]

classes = ("Ship", "General float", "Fishing net buoy")

fig, ax = plt.subplots(figsize=(12, 12))

for (l, t, r, b), c in zip(dataset[i]['gt_bboxes'].data, dataset[i]['gt_labels'].data):
    
    rect = mpl.patches.Rectangle((l, t), r-l, b-t, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    
    ax.text(x=l, y=t, s=classes[c], fontsize=10, color='white', backgroundcolor='red')

plt.imshow(img)
plt.show()

In [7]:
model = build_detector(cfg.model)
model.init_weights()

2022-05-27 18:38:43,290 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth
2022-05-27 18:38:43,407 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-05-27 18:38:43,429 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-05-27 18:38:43,435 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-05-27 18:38:43,536 - mmcv - INFO - 
backbone.patch_embed.projection.weight - torch.Size([96, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2022-05-27 18:38:43,537 - mmcv - INFO - 
backbo

In [8]:
train_detector(model, dataset, cfg, distributed=False, validate=True)

/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2022-05-27 18:38:47,961 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...


2022-05-27 18:38:48,271 - mmdet - INFO - Start running, host: root@4a37eb830c7b, work_dir: /opt/ml/mmdetection/work_dirs/all-seaing-eye
2022-05-27 18:38:48,272 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(ABOVE_NORMAL) Fp16OptimizerHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) I

Done (t=0.22s)
creating index...
index created!


wandb: Currently logged in as: jaeh8nkim. Use `wandb login --relogin` to force relogin


...

Loading and preparing results...
DONE (t=0.59s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=20.67s).
Accumulating evaluation results...


2022-05-28 03:48:36,546 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.333
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.443
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.389
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.162
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.615
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.348
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.348
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.348
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.175
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=2.07s).


2022-05-28 03:49:28,184 - mmdet - INFO - Epoch [14][50/2614]	lr: 1.000e-06, eta: 23:13:20, time: 1.030, data_time: 0.161, memory: 12662, loss_rpn_cls: 0.0011, loss_rpn_bbox: 0.0134, loss_cls: 0.0441, acc: 98.1965, loss_bbox: 0.1138, loss: 0.1725, grad_norm: 1.5913
2022-05-28 03:50:11,251 - mmdet - INFO - Epoch [14][100/2614]	lr: 1.000e-06, eta: 23:12:37, time: 0.861, data_time: 0.020, memory: 12662, loss_rpn_cls: 0.0010, loss_rpn_bbox: 0.0141, loss_cls: 0.0432, acc: 98.1946, loss_bbox: 0.1137, loss: 0.1719, grad_norm: 1.5913
2022-05-28 03:50:55,100 - mmdet - INFO - Epoch [14][150/2614]	lr: 1.000e-06, eta: 23:11:55, time: 0.877, data_time: 0.027, memory: 12662, loss_rpn_cls: 0.0010, loss_rpn_bbox: 0.0137, loss_cls: 0.0442, acc: 98.1665, loss_bbox: 0.1157, loss: 0.1747, grad_norm: 1.6396
2022-05-28 03:51:38,562 - mmdet - INFO - Epoch [14][200/2614]	lr: 1.000e-06, eta: 23:11:12, time: 0.869, data_time: 0.020, memory: 12662, loss_rpn_cls: 0.0011, loss_rpn_bbox: 0.0144, loss_cls: 0.0464, ac

KeyboardInterrupt: 